In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import collections
import random


class DQNAgent(torch.nn.Module):
    def __init__(self, params):
        super().__init__()
        self.first_layer = params["first_layer_size"]
        self.second_layer = params["second_layer_size"]
        self.third_layer = params["third_layer_size"]
        self.weights = params["weights_path"]
        self.load_weights = params['load_weights']
        self.memory = collections.deque(maxlen=params['memory_size'])
        self.learning_rate = params["learning_rate"]
        self.gamma = 0.9
        self.epsilon = 1
        self.agent_target = 1
        self.agent_predict = 0
        self.optimizer = None

        self.network()

    def hot_encode(self, cards, length=54):
        try:
            tensor = torch.zeros(length)
            tensor[cards] = 1.0
            return tensor
        except IndexError:
            print(cards)
            tensor = torch.zeros(length)
            raise ValueError("Let see this index error thingy")
            return tensor

    def network(self):
        self.f1 = nn.Linear(117, self.first_layer)
        self.f2 = nn.Linear(self.first_layer, self.second_layer)
        self.f3 = nn.Linear(self.second_layer, self.third_layer)
        self.f4 = nn.Linear(self.third_layer, 60)

        if self.load_weights:
            self.model = self.load_state_dict(torch.load(self.weights))
            print("weights loaded")


    def forward(self, x):
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = F.relu(self.f3(x))
        x = F.softmax(self.f4(x), dim=-1)

        return x

    # def set_reward(self, closer, outcome):
    #     self.reward = 0
    #     if closer != 0:
    #         self.reward = closer

    #     if outcome == 'win':
    #         self.reward = 10
    #     if outcome == 'lose':
    #         self.reward == -10

    #     return self.reward

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay_new(self, memory, batch_size):
        if len(memory) > batch_size:
            minibatch = random.sample(memory, batch_size)
        else:
            minibatch = memory

        for state, action, reward, next_state, done in minibatch:
            self.train()
            torch.set_grad_enabled(True)

            target = reward
            next_state_tensor = torch.tensor(np.expand_dims(
                next_state, 0), dtype=torch.float32, requires_grad=True)
            state_tensor = torch.tensor(np.expand_dims(
                state, 0), dtype=torch.float32, requires_grad=True)

            if not done:
                target = reward + self.gamma * \
                    torch.max(self.forward(next_state_tensor[0]))

            output = self.forward(state_tensor)
            target_f = output.clone()
            target_f[0][np.argmax(action)] = target
            target_f.detach()
            self.optimizer.zero_grad()
            loss = F.mse_loss(output, target_f)
            loss.backward()
            self.optimizer.step()

    def train_short_memory(self, state, action, reward, next_state, done):
        self.train()
        torch.set_grad_enabled(True)

        target = reward
        next_state_tensor = torch.tensor(np.expand_dims(
            next_state, 0), dtype=torch.float32, requires_grad=True)
        state_tensor = torch.tensor(np.expand_dims(
            state, 0), dtype=torch.float32, requires_grad=True)

        if not done:
            target = reward + self.gamma * \
                torch.max(self.forward(next_state_tensor[0]))

        output = self.forward(state_tensor)
        target_f = output.clone()
        target_f[0][np.argmax(action)] = target
        target_f.detach()
        self.optimizer.zero_grad()
        loss = F.mse_loss(output, target_f)
        loss.backward()
        self.optimizer.step()

    def simple_env(self, agent_hand, top_card, turn, cardless, action=-1):
        hand = self.hot_encode(agent_hand)
        top = self.hot_encode([top_card])
        actions = torch.zeros(7)
        turn_t = torch.tensor([turn], requires_grad=False)
        cardless_t = torch.tensor([cardless], requires_grad=False)
        if action != -1:
            actions[action] = 1

        env_space = torch.cat([hand, top, actions, cardless_t, turn_t], dim=0)
        return env_space

    # def get_env_space(self, agent_hand, top_card, wastes, deck_size, actions, player_turn, poss_moves):
        print(agent_hand, top_card, wastes, deck_size, actions, player_turn)
        hand = hot_encode(agent_hand)
        top = hot_encode([top_card])
        waste = hot_encode(wastes)
        deck = torch.zeros([55])
        deck[0] = deck_size/54
        action = hot_encode(actions)
        player_turn = hot_encode(player_turn)

        env_space = torch.stack([hand, top, waste, deck, action, player_turn])

        env_space_sq = torch.cat(
            [env_space.unsqueeze(dim=0), torch.zeros(1, 10, 55)], dim=1)
        action_space = torch.zeros(32, 16, 55)

        print("size", env_space_sq.size())
        action_space[31, 0, 54] = 1.0

        for i, move in enumerate(poss_moves):
            for j, card in enumerate(poss_moves[i]):
                action_space[i, j, card] = 1.0

        combined_state = torch.cat([env_space_sq, action_space], dim=0)
        return combined_state

    # def get_action_space(self, poss_moves):

    #     return action_space


In [5]:
import random
import math
import torch
import numpy as np
from DQN import DQNAgent
import torch.optim as optim
from typing import Type
from cards import cards as cs

params = dict()

params["first_layer_size"] = 1024
params["second_layer_size"] = 512
params["third_layer_size"] = 256
params["learning_rate"] = 0.99
params["memory_size"] = 2500
params["load_weights"] = False
params['train'] = True
params["epsilon_decay_linear"] = 0.001
params["episodes"] = 10
params["batch_size"] = 200


params1 = params.copy()
params2 = params.copy()

params1["weights_path"] = "weights/agent1/weights.h5"
params2["weights_path"] = "weights/agent2/weights.h5"

questions = [28, 29, 30, 31, 51, 50, 49, 48, 47, 46, 45, 44]
aces = [0, 1, 2, 3]
punishers = [4, 5, 6, 7, 8, 9, 10, 11, 52, 53]
all_cards_without_jokers = list(range(52))


def to_cs(n_l):
    h = []
    for n in n_l:
        h.append(cs[n])
    return h


class Player():
    def __init__(self, game, index) -> None:
        self.hand = []
        self.build = []
        self.game = game
        self.asking = False
        self.index = index
        self.reward = 0

    def can_complete(self):
        if len(self.build) == 0 or self.build[-1] in questions:
            return False
        else:
            return True

    def waste_card(self, card):
        white = self.game.white_list(build=self.build)
        if card in white:
            self.build += [card]
            # print(f"build:{ self.build} card: {card}")
            self.hand = list(filter(lambda x: x != card, self.hand))
            self.reward  = 1
        else:
            # print(f"build:{ self.build} card: {card}")
            self.reward = -1
            # wrong move

    def pick_cards(self, game):
        game.pick(player=self)
        self.game.new_turn()
        self.build.clear()
        self.reward = ((len(self.hand)-4) / 2)* -1 if len(self.hand)> 6 else 0

    def complete_build(self, game):
        if len(self.build) == 0:
            self.reward = -1
            return

        game.waste(self)
        

        if len(self.hand) == 0 and self.build[-1] not in questions + aces + punishers and game.card_less == False:
            self.reward = 10
            game.complete = True
            self.build = []
            return
        elif len(self.hand) == 0 and (self.build[-1] in questions+aces+punishers or game.card_less == True):
            game.card_less = True

        self.build.clear()
        self.reward = 3
        if not self.asking:
            self.game.new_turn()

    def choose_flower(self, flower, game):
        if self.asking:
            game.action = flower
            game.new_turn()
            self.asking = False
        else:
            self.reward = -1

    def do_move(self, move, game):
        self.reward = 0
        if move < 54:
            self.waste_card(move)
        elif move > 53 and move < 58:
            self.choose_flower(move-54, game)
        elif move == 58:
            self.complete_build(game)
        elif move == 59:
            self.pick_cards(game)


class Game():
    def __init__(self) -> None:
        self.complete = True
        self.wastes = []
        self.deck = []
        self.top_card = None
        self.action = -1
        self.turn = 0
        self.card_less = False

    def new_turn(self):
        self.turn = (self.turn+1) % 2

    def waste(self, player):
        self.action = -1
        top = player.build[-1]
        if self.action > 3 and top < 4:
            aces_in_build = list(filter(lambda x: x < 4, player.build))

            if len(aces_in_build) < 2:
                top = self.wastes[-1]
                self.turn = (self.turn+1) % 2
                self.wastes = self.wastes[0:-1] + player.build + self.wastes[-1:]
            else:
                player.asking = True
        elif top < 4:
            player.asking = True
            self.wastes += player.build
        else:
            self.wastes += player.build

        self.top_card = self.wastes[-1]

        if self.top_card in punishers:
            if math.floor(self.top_card/4) == 2:
                self.action = 4
            elif math.floor(self.top_card/4) == 3:
                self.action = 5
            elif self.top_card in [52, 53]:
                self.action = 6

    def pick(self, player):
        pick_num = 1
        if self.action == 4:
            pick_num = 2
            self.action = -1
        elif self.action == 5:
            pick_num = 3
            self.action = -1
        elif self.action == 6:
            pick_num = 5
            self.action = -1

        if len(self.deck) < pick_num:
            self.deck += self.wastes[0:-1]
            self.wastes = self.wastes[-1:]
            self.top_card = self.wastes[-1]

        picked_cards = random.sample(self.deck, pick_num)
        self.deck = list(filter(lambda x: x not in picked_cards, self.deck))

        player.hand += picked_cards

    def white_list(self, build=[]):
        if self.action != -1:
            if self.action < 4:
                white = list(filter(lambda x: x %
                             4 == self.action, all_cards_without_jokers))
                if self.action < 2:
                    white += [52]
                else:
                    white += [53]
                return white + aces
            elif self.action == 4:
                return [4, 5, 6, 7] + aces
            elif self.action == 5:
                return [8, 9, 10, 11] + aces
            elif self.action == 6:
                return [52, 53] + aces

        if len(build) == 0:
            if self.top_card == 52:
                white = list(filter(lambda x: x %
                             4 < 2, all_cards_without_jokers))
                white += aces + [52, 53]
            elif self.top_card == 53:
                white = list(filter(lambda x: x %
                             4 > 1, all_cards_without_jokers))
                white += aces + [52, 53]
            else:
                white = list(filter(lambda x: x % 4 == self.top_card % 4 or math.floor(
                    x/4) == math.floor(self.top_card/4), all_cards_without_jokers))
                if self.top_card % 4 < 2:
                    white += [52]
                else:
                    white += [53]
            return white
        else:
            last = build[-1]
            if last in questions:
                white = list(filter(lambda x: x % 4 == last % 4 or math.floor(
                    x/4) == math.floor(last/4), all_cards_without_jokers))
                if self.top_card % 4 < 2:
                    white += [52]
                else:
                    white += [53]
                return white + aces
            elif last == 52 or last == 53:
                return [52, 53]
            else:
                white = list(filter(lambda x: math.floor(
                    x/4) == math.floor(last/4), all_cards_without_jokers))
                return white


def initialize_game(game, players):
    game.deck = list(range(54))

    for player in players:
        player.hand = random.sample(game.deck, 4)

        game.deck = list(filter(lambda x: x not in player.hand, game.deck))

    poss_top = list(
        filter(lambda x: x not in questions+aces+punishers, game.deck))
    game.top_card = random.choice(poss_top)
    game.wastes.append(game.top_card)

    game.deck = list(filter(lambda x: x != game.top_card, game.deck))
    game.complete = False


game = Game()
player1 = Player(game=game, index=0)
player2 = Player(game=game, index=1)


def play(player: Type[Player], agent: Type[DQNAgent], game: Type[Game], params: any, opponent: Type[Player], opp_agent: Type[DQNAgent]):
    turn = 1
    if (player.index != game.turn):
        return

    cardless = 1 if game.card_less == True else 0
    env_space = agent.simple_env(
        agent_hand=player.hand, top_card=game.top_card, turn=turn, cardless=cardless, action=game.action)

    if random.uniform(0, 1) < agent.epsilon:
        prediction = torch.rand(60)
    else:
        with torch.no_grad():
            state = torch.tensor(env_space.clone().detach().reshape(1, 117), dtype=torch.float32)
            prediction = agent(state)
            print(prediction)

    if player.asking:
        mask = torch.cat([torch.zeros(54), torch.tensor(
            [1, 1, 1, 1], requires_grad=False), torch.zeros(2)], dim=0)
    else:
        complete = 1 if player.can_complete() else 0
        asks = [0, 0, 0, 0]
        pick = 0 if (len(player.build) >
                     0 and player.build[-1] not in questions) else 1
        mask = torch.cat([agent.hot_encode(player.hand), torch.tensor(
            asks, requires_grad=False), torch.tensor([complete, pick], requires_grad=False)])

    move = prediction.clone().detach()*mask

    do_move = np.argmax(move).cpu().detach().numpy()

    print(f"player {1 if player == player1 else 2}")
    # print(f"hand {to_cs(player.hand)}")

    if torch.all(torch.eq(move, 0)).item():
        player.reward = -3
    else:
        player.do_move(move=do_move.item(), game=game)

    if game.complete:
        opponent.reward = -10
        op_env = agent.simple_env(
            agent_hand=opponent.hand, top_card=game.top_card, turn=turn, cardless=game.card_less, action=game.action
        )

        opp_agent.remember(state=op_env, action=torch.zeros(60),
                           reward=opponent.reward, next_state=op_env, done=game.complete)
        if params['train']:
            opp_agent.replay_new(memory=opp_agent.memory,
                                 batch_size=params['batch_size'])
            model_weights = opp_agent.state_dict()
            if player == player1:
                torch.save(model_weights, params2["weights_path"])
            else:
                torch.save(model_weights, params1["weights_path"])

    final_move = np.eye(60)[np.argmax(move).numpy()]

    # print(f"topCard: {cs[game.top_card]}")
    # print(f"move:{cs[do_move]}")
    print(f"reward {player.reward}")
    # print(f"action:{game.action}")

    if (player.index != game.turn):
        turn = 0

    next_state = agent.simple_env(agent_hand=player.hand, top_card=game.top_card,
                                  turn=turn, cardless=game.card_less, action=game.action)

    agent.remember(state=env_space, action=final_move,
                   reward=player.reward, next_state=next_state, done=game.complete)
    


def run():
    agent1 = DQNAgent(params=params1)
    agent1.optimizer = optim.Adam(
        agent1.parameters(), weight_decay=0, lr=params1['learning_rate'])
    agent2 = DQNAgent(params=params2)
    agent2.optimizer = optim.Adam(
        agent2.parameters(), weight_decay=0, lr=params2['learning_rate'])
    games_count = 0
    steps = 0
    def replay(agent,  params):
            agent.replay_new(memory=agent.memory, batch_size=params['batch_size'])
            model_weights = agent.state_dict()
            torch.save(model_weights, params["weights_path"])
    
    while games_count < params['episodes']:
        if game.complete:
            steps = 0
            initialize_game(game=game, players=[player1, player2])
            print("\nhands")

            print(to_cs(player1.hand))
            print(to_cs(player2.hand))

            print("\n top card")
            print(cs[game.top_card])
        while not game.complete:
            if game.turn == 0:
                if not params1['train']:
                    agent1.epsilon = 0.01
                else:
                    agent1.epsilon = 1 - (steps * params1["epsilon_decay_linear"])

                play(player=player1, game=game, agent=agent1,
                     params=params1, opponent=player2, opp_agent=agent2)
            elif game.turn == 1:
                if not params2['train']:
                    agent2.epsilon = 0.01
                else:
                    agent2.epsilon = 1 - \
                        (games_count * params1["epsilon_decay_linear"])
                play(player=player2, game=game, agent=agent2,
                     params=params2, opponent=player1, opp_agent=agent1)

            steps += 1
            if steps%400 == 0:
                replay(agent1, params1)
                replay(agent2, params2)
                
            print(f"game: {games_count}.  step: {steps} turn: {game.turn}")
        
        
        if game.complete:
            games_count += 1 
            if params['train']:
                replay(agent1, params1)
                replay(agent2, params2)

        


run()



hands
['6_of_hearts', '7_of_hearts', '5_of_diamonds', '10_of_hearts']
['9_of_diamonds', '8_of_hearts', '7_of_spades', '3_of_diamonds']

 top card
5_of_hearts
player 1
reward 1
game: 0.  step: 1 turn: 0
player 1
reward 3
game: 0.  step: 2 turn: 1
player 2
reward 0
game: 0.  step: 3 turn: 0
player 1
reward 1
game: 0.  step: 4 turn: 0
player 1
reward -1
game: 0.  step: 5 turn: 0
player 1
reward -1
game: 0.  step: 6 turn: 0
player 1
reward -1
game: 0.  step: 7 turn: 0
player 1
reward -1
game: 0.  step: 8 turn: 0
player 1
reward -1
game: 0.  step: 9 turn: 0
tensor([[0.0163, 0.0162, 0.0170, 0.0157, 0.0158, 0.0167, 0.0175, 0.0167, 0.0163,
         0.0175, 0.0162, 0.0179, 0.0164, 0.0161, 0.0158, 0.0161, 0.0169, 0.0158,
         0.0167, 0.0169, 0.0166, 0.0173, 0.0167, 0.0158, 0.0162, 0.0165, 0.0168,
         0.0161, 0.0175, 0.0165, 0.0162, 0.0162, 0.0168, 0.0159, 0.0170, 0.0173,
         0.0160, 0.0164, 0.0165, 0.0175, 0.0157, 0.0163, 0.0167, 0.0175, 0.0169,
         0.0171, 0.0171, 0.0173, 0.

/tmp/ipykernel_6097/2759861331.py:267: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(env_space.clone().detach().reshape(1, 117), dtype=torch.float32)


player 2
reward -1
game: 0.  step: 86 turn: 1
player 2
reward -1
game: 0.  step: 87 turn: 1
player 2
reward -1
game: 0.  step: 88 turn: 1
player 2
reward -1
game: 0.  step: 89 turn: 1
player 2
reward 3
game: 0.  step: 90 turn: 1
player 2
reward 0
game: 0.  step: 91 turn: 0
player 1
reward 0
game: 0.  step: 92 turn: 1
player 2
reward -1
game: 0.  step: 93 turn: 1
player 2
reward -1
game: 0.  step: 94 turn: 1
player 2
reward 1
game: 0.  step: 95 turn: 1
player 2
reward -1
game: 0.  step: 96 turn: 1
player 2
reward -1
game: 0.  step: 97 turn: 1
player 2
reward -1
game: 0.  step: 98 turn: 1
player 2
reward 3
game: 0.  step: 99 turn: 0
player 1
reward -1
game: 0.  step: 100 turn: 0
player 1
reward 1
game: 0.  step: 101 turn: 0
player 1
reward 3
game: 0.  step: 102 turn: 0
player 1
reward 0
game: 0.  step: 103 turn: 1
player 2
reward -2.0
game: 0.  step: 104 turn: 0
player 1
reward 0
game: 0.  step: 105 turn: 1
player 2
reward -1
game: 0.  step: 106 turn: 1
player 2
reward -1
game: 0.  step:

game: 0.  step: 232 turn: 1
player 2
reward -1
game: 0.  step: 233 turn: 1
player 2
reward -1
game: 0.  step: 234 turn: 1
player 2
reward -1
game: 0.  step: 235 turn: 1
player 2
reward -1
game: 0.  step: 236 turn: 1
player 2
reward -5.0
game: 0.  step: 237 turn: 0
player 1
reward -1
game: 0.  step: 238 turn: 0
player 1
reward -1
game: 0.  step: 239 turn: 0
player 1
reward -1
game: 0.  step: 240 turn: 0
player 1
reward -1
game: 0.  step: 241 turn: 0
player 1
reward 1
game: 0.  step: 242 turn: 0
player 1
reward -1
game: 0.  step: 243 turn: 0
tensor([[0.0164, 0.0161, 0.0169, 0.0155, 0.0159, 0.0167, 0.0173, 0.0167, 0.0162,
         0.0178, 0.0163, 0.0177, 0.0162, 0.0162, 0.0158, 0.0163, 0.0170, 0.0157,
         0.0167, 0.0170, 0.0165, 0.0173, 0.0168, 0.0157, 0.0163, 0.0163, 0.0168,
         0.0162, 0.0175, 0.0164, 0.0164, 0.0163, 0.0171, 0.0160, 0.0171, 0.0171,
         0.0161, 0.0164, 0.0165, 0.0175, 0.0156, 0.0163, 0.0167, 0.0173, 0.0170,
         0.0172, 0.0170, 0.0173, 0.0162, 0.0179, 

tensor([[0.0164, 0.0163, 0.0171, 0.0158, 0.0159, 0.0167, 0.0175, 0.0163, 0.0160,
         0.0174, 0.0163, 0.0179, 0.0165, 0.0159, 0.0158, 0.0162, 0.0169, 0.0158,
         0.0165, 0.0170, 0.0164, 0.0174, 0.0167, 0.0155, 0.0163, 0.0163, 0.0165,
         0.0162, 0.0176, 0.0165, 0.0164, 0.0163, 0.0172, 0.0160, 0.0174, 0.0171,
         0.0161, 0.0163, 0.0165, 0.0175, 0.0155, 0.0162, 0.0167, 0.0173, 0.0169,
         0.0170, 0.0169, 0.0174, 0.0162, 0.0178, 0.0156, 0.0170, 0.0175, 0.0176,
         0.0172, 0.0173, 0.0158, 0.0174, 0.0167, 0.0175]])
player 1
reward -1
game: 0.  step: 394 turn: 0
tensor([[0.0164, 0.0163, 0.0171, 0.0158, 0.0159, 0.0167, 0.0175, 0.0163, 0.0160,
         0.0174, 0.0163, 0.0179, 0.0165, 0.0159, 0.0158, 0.0162, 0.0169, 0.0158,
         0.0165, 0.0170, 0.0164, 0.0174, 0.0167, 0.0155, 0.0163, 0.0163, 0.0165,
         0.0162, 0.0176, 0.0165, 0.0164, 0.0163, 0.0172, 0.0160, 0.0174, 0.0171,
         0.0161, 0.0163, 0.0165, 0.0175, 0.0155, 0.0162, 0.0167, 0.0173, 0.0169,
   

...

reward -3
game: 0.  step: 6648 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6649 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6650 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  st

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6686 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6687 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6688 turn: 0
tensor([[0., 0., 0., 0

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6727 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6728 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6729 turn: 0
tensor([[0., 0., 0., 0

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6768 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6769 turn: 0
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
player 1
reward -3
game: 0.  step: 6770 turn: 0
tensor([[0., 0., 0., 0

KeyboardInterrupt: 